# FD-LLM Explainer - Google Colab GPU Edition

This notebook generates LLM explanations for your trained classifier predictions.

**Requirements:**
- Google Colab Pro (recommended for faster GPU)
- Trained model predictions uploaded to Google Drive
- GPU runtime (T4, V100, or A100)

**Before running:**
1. Upload `predictions_for_colab.parquet` to your Google Drive
2. Upload the `explainer/` folder to your Google Drive
3. Update the paths in Cell 3 (Configuration) below
4. Runtime → Change runtime type → GPU

**Estimated time:**
- 100 windows: ~5-10 minutes
- 1000 windows: ~50-90 minutes  
- 5000 windows: ~3-5 hours on T4


## Step 1: Check GPU and Mount Drive


In [1]:
# Check GPU availability
print("Checking GPU...")
!nvidia-smi


Checking GPU...
Sun Oct 12 19:18:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P0             34W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------------------

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✓ Google Drive mounted")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Google Drive mounted


## Step 2: Install Dependencies


In [3]:
# Install required packages (takes ~2 minutes)
print("Installing dependencies...")
%pip install -q torch transformers peft bitsandbytes accelerate
%pip install -q pandas numpy scikit-learn pyarrow tqdm
print("✓ All dependencies installed")


Installing dependencies...
✓ All dependencies installed


## Step 3: Configuration

**IMPORTANT: Update these paths to match your Google Drive structure!**

### LLM Model Options

**Default: DeepSeek-R1-Distill-Llama-8B** (Recommended)
- ✅ **Open-access** - no authentication required
- ✅ **Excellent reasoning** - strong at technical analysis
- ✅ **8B parameters** - fits in T4 GPU with 4-bit quantization
- ✅ **Fast** - optimized distilled model

**Alternatives:**
- `mistralai/Mistral-7B-Instruct-v0.2` - Slightly faster, 7B model
- `meta-llama/Llama-3.1-8B-Instruct` - Requires HuggingFace access approval


In [4]:
import sys

# Add FD-LLM to Python path
sys.path.append('/content/drive/MyDrive/fd-llm')

# Configuration
CONFIG = {
    # Input: predictions from trained classifier
    'predictions_file': '/content/drive/MyDrive/fd-llm/outputs/exp_full_dataset/predictions_for_colab.parquet',

    # Output: explanations
    'output_file': '/content/drive/MyDrive/fd-llm/outputs/exp_full_dataset/explanations.jsonl',

    # LLM settings
    'model_name': 'mistralai/Mistral-7B-Instruct-v0.2',  # Open-access, excellent reasoning
    # Alternatives:
    #   'meta-llama/Llama-3.1-8B-Instruct' (requires HF access approval)
    'load_in_4bit': True,  # Use 4-bit quantization (fits in T4 16GB)
    'self_consistency_k': 5,  # Number of explanations to generate and vote
    'temperature': 0.8,

    # Processing (set to None to process all windows)
    'max_samples': None,  # Change to 10 for quick testing
}

print("Configuration:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")


Configuration:
  predictions_file: /content/drive/MyDrive/fd-llm/outputs/exp_full_dataset/predictions_for_colab.parquet
  output_file: /content/drive/MyDrive/fd-llm/outputs/exp_full_dataset/explanations.jsonl
  model_name: mistralai/Mistral-7B-Instruct-v0.2
  load_in_4bit: True
  self_consistency_k: 5
  temperature: 0.8
  max_samples: None


## Step 4: Load LLM

This takes 2-3 minutes to download and load the model.


In [5]:
from explainer.llm_setup import LLMExplainer

print("Loading LLM (this may take 2-3 minutes)...")
print(f"Model: {CONFIG['model_name']}")
print(f"Quantization: {'4-bit' if CONFIG['load_in_4bit'] else 'fp16'}")

llm_explainer = LLMExplainer(
    model_name=CONFIG['model_name'],
    load_in_4bit=CONFIG['load_in_4bit'],
    temperature=CONFIG['temperature']
)

print("\n✓ LLM loaded and ready!")


Loading LLM (this may take 2-3 minutes)...
Model: mistralai/Mistral-7B-Instruct-v0.2
Quantization: 4-bit
Loading LLM: mistralai/Mistral-7B-Instruct-v0.2
4-bit quantization: True
Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model loaded successfully
Device map: {'': 0}

✓ LLM loaded and ready!


## Step 5: Load Predictions


In [16]:
import pandas as pd
import json

# Load predictions
print(f"Loading predictions from: {CONFIG['predictions_file']}")
pred_df = pd.read_parquet(CONFIG['predictions_file'])

# Limit samples if requested (for testing)
if CONFIG['max_samples']:
    print(f"Limiting to {CONFIG['max_samples']} samples for testing")
    pred_df = pred_df.head(CONFIG['max_samples'])

print(f"\n✓ Loaded {len(pred_df)} predictions")
print(f"Columns: {list(pred_df.columns)}")
print("\nFirst few predictions:")
print(pred_df[['window_id', 'prediction', 'confidence']].head())


Loading predictions from: /content/drive/MyDrive/fd-llm/outputs/exp_full_dataset/predictions_for_colab.parquet

✓ Loaded 5236 predictions
Columns: ['window_id', 'prediction', 'confidence', 'features']

First few predictions:
   window_id            prediction  confidence
0          0              Dilution    0.746451
1          1              Dilution    0.456028
2          2  Settling/Segregation    0.775276
3          3  Settling/Segregation    0.780793
4          4  Settling/Segregation    0.786817


In [17]:
# Check if the fix is loaded
import sys
print("Python path:")
for p in sys.path:
    if 'fd-llm' in p:
        print(f"  ✓ {p}")

# Check the actual file content in memory
from explainer import prompt_templates
import inspect

# Get the source code of the create_explanation_prompt function
source = inspect.getsource(prompt_templates.create_explanation_prompt)

# Check if the fix is present (look for the 'fmt' helper function)
if 'def fmt(key, default' in source:
    print("\n✅ FIX IS LOADED - The helper function 'fmt' is present")
    print("The updated code is being used!")
else:
    print("\n❌ OLD CODE STILL LOADED - The fix is NOT present")
    print("You need to restart the runtime and reimport")

# Also check the file on disk
print("\n" + "="*60)
print("Checking file on Google Drive:")
print("="*60)

try:
    with open('/content/drive/MyDrive/fd-llm/explainer/prompt_templates.py', 'r') as f:
        file_content = f.read()

    if 'def fmt(key, default' in file_content:
        print("✅ File on Drive has the fix")
    else:
        print("❌ File on Drive does NOT have the fix - re-upload needed")

    # Show a snippet around the fix
    lines = file_content.split('\n')
    for i, line in enumerate(lines):
        if 'def fmt(key' in line:
            print(f"\nFound fix at line {i+1}:")
            for j in range(max(0, i-2), min(len(lines), i+10)):
                print(f"  {j+1}: {lines[j]}")
            break

except FileNotFoundError:
    print("❌ File not found on Drive - check the path")

Python path:
  ✓ /content/drive/MyDrive/fd-llm
  ✓ /content/drive/MyDrive/fd-llm

✅ FIX IS LOADED - The helper function 'fmt' is present
The updated code is being used!

Checking file on Google Drive:
✅ File on Drive has the fix

Found fix at line 70:
  68:     
  69:     # Helper function to format numeric values
  70:     def fmt(key, default=0.0, decimals=3):
  71:         val = features.get(key, default)
  72:         if val is None or (isinstance(val, str) and val == 'N/A'):
  73:             return 'N/A'
  74:         try:
  75:             return f"{float(val):.{decimals}f}"
  76:         except (ValueError, TypeError):
  77:             return 'N/A'
  78:     
  79:     # Format sensor data section


In [18]:
#relaxing the validation
# Patch the validation to be more lenient
import sys
sys.path.insert(0, '/content/drive/MyDrive/fd-llm')

# Reload the module with a monkey patch
from explainer import prompt_templates

def relaxed_validate(explanation):
    """Simplified validation - just check for digits in evidence."""
    required_keys = ['final_diagnosis', 'confidence', 'evidence', 'cross_checks', 'recommended_actions']

    for key in required_keys:
        if key not in explanation:
            return False, f"Missing required key: {key}"

    if not isinstance(explanation['final_diagnosis'], str):
        return False, "final_diagnosis must be a string"

    if not isinstance(explanation['confidence'], (int, float)):
        return False, "confidence must be a number"

    if not (0 <= explanation['confidence'] <= 1):
        return False, "confidence must be between 0 and 1"

    if not isinstance(explanation['evidence'], list):
        return False, "evidence must be a list"

    if len(explanation['evidence']) < 3:
        return False, "evidence must contain at least 3 claims"

    # SIMPLIFIED: Just count claims with ANY digit
    numeric_claims = sum(1 for claim in explanation['evidence']
                        if any(c.isdigit() for c in str(claim)))

    if numeric_claims < 2:  # Relaxed from 3 to 2
        return False, f"evidence should reference sensor data (found {numeric_claims} claims with numbers)"

    return True, "valid"

# Replace the validation function
prompt_templates.validate_explanation_format = relaxed_validate

print("✓ Validation patched - now more lenient with evidence requirements")
print("Re-run the explanation generation cells")

✓ Validation patched - now more lenient with evidence requirements
Re-run the explanation generation cells


## Step 6: Generate Explanations

**This is the main processing step. Progress is saved incrementally to Drive.**

Estimated time:
- 10 windows: ~1-2 minutes
- 100 windows: ~5-10 minutes  
- 1000 windows: ~50-90 minutes
- 5236 windows: ~3-5 hours on T4


In [20]:
# Debug: Check what the LLM is actually generating
from explainer.llm_setup import LLMExplainer
from explainer.prompt_templates import create_explanation_prompt, SYSTEM_PROMPT, validate_explanation_format
from explainer.self_consistency import parse_json_from_text

# Get one sample to test
test_features = json.loads(pred_df.iloc[0]['features'])
test_pred = pred_df.iloc[0]['prediction']
test_conf = pred_df.iloc[0]['confidence']

# Generate prompt
user_prompt = create_explanation_prompt(test_features, test_pred, test_conf)

print("="*60)
print("PROMPT BEING SENT TO LLM:")
print("="*60)
print(user_prompt[:1000] + "..." if len(user_prompt) > 1000 else user_prompt)

print("\n" + "="*60)
print("GENERATING RESPONSE...")
print("="*60)

# Generate response
response = llm_explainer.explain(
    system_prompt=SYSTEM_PROMPT,
    user_prompt=user_prompt,
    temperature=0.8
)

print("\nRAW LLM RESPONSE:")
print("="*60)
print(response)
print("="*60)

# Try to parse
print("\nPARSING JSON...")
parsed = parse_json_from_text(response)

if parsed:
    print("✓ JSON parsed successfully")
    print(json.dumps(parsed, indent=2))

    # Validate
    print("\nVALIDATING FORMAT...")
    is_valid, error_msg = validate_explanation_format(parsed)

    if is_valid:
        print(f"✓ Validation passed: {error_msg}")
    else:
        print(f"✗ Validation failed: {error_msg}")
        print("\nDEBUGGING:")
        print(f"  - Has final_diagnosis: {'final_diagnosis' in parsed}")
        print(f"  - Has confidence: {'confidence' in parsed}")
        print(f"  - Has evidence: {'evidence' in parsed}")
        if 'evidence' in parsed:
            print(f"  - Evidence count: {len(parsed['evidence'])}")
            print(f"  - Evidence items:")
            for i, ev in enumerate(parsed['evidence']):
                has_digit = any(c.isdigit() for c in str(ev))
                has_op = any(op in str(ev) for op in ['>', '<', '=', '±', '%'])
                print(f"    {i+1}. '{ev[:80]}...' [digit:{has_digit}, op:{has_op}]")
else:
    print("✗ Failed to parse JSON from response")
    print("The LLM may not be outputting valid JSON")

PROMPT BEING SENT TO LLM:
PRIMARY CLASSIFIER PREDICTION:
  Diagnosis: Dilution
  Confidence: 0.746

FLOW MEASUREMENTS:
  Mean Flow: nan m³/s
  Flow Std Dev: nan m³/s
  Coefficient of Variation: nan (threshold: 0.15)
  Zero-Flow Events: 0
  Rate of Change: nan m³/s per min
  Flow Stability: nan

DENSITY & SOLIDS:
  Density Mean: 1009.6 kg/m³
  Density Std Dev: 9.5 kg/m³
  Density Trend: -65.00 kg/m³ per 5min
  Density Spikes (>3σ): 1
  SG Mean: 1.010
  SG Target Deviation: nan (threshold: ±0.03)
  Percent Solids: 0.0%

PROCESS VARIABLES:
  Pressure Mean: 0.0 kPa
  Pressure Variability: 0.000 (threshold: 0.1)
  Pressure Spikes: 0
  Pressure-Flow Correlation: 0.000
  Temperature: 0.0°C
  DV (Particle Size): 0.0 μm
  DV Drift: 0.0 μm (segregation threshold: 5.0 μm)

MASS BALANCE:
  Measured Mass Flow: nan kg/s
  Calculated (Flow × Density): nan kg/s

TASK:
Analyze the sensor data and provide a structured explanation for the fault diagnosis.
Consider the primary classifier's prediction but 

In [19]:
from explainer.self_consistency import explain_with_self_consistency
from datetime import datetime
from pathlib import Path
from tqdm import tqdm

# Create output directory
output_path = Path(CONFIG['output_file'])
output_path.parent.mkdir(parents=True, exist_ok=True)

print(f"Generating explanations for {len(pred_df)} windows...")
print(f"Output will be saved to: {output_path}")
print(f"Progress is saved incrementally (safe to interrupt)\n")

explanations = []

with open(output_path, 'w') as f:
    for idx, row in tqdm(pred_df.iterrows(), total=len(pred_df), desc="Generating explanations"):
        # Parse features
        try:
            features = json.loads(row['features'])
        except json.JSONDecodeError:
            print(f"Skipping window {row['window_id']} due to invalid JSON in features.")
            continue

        prediction = row['prediction']
        confidence = row['confidence']

        # Generate explanation with self-consistency
        try:
            explanation = explain_with_self_consistency(
                llm_explainer=llm_explainer,
                features=features,
                prediction=prediction,
                confidence=confidence,
                k=CONFIG['self_consistency_k'],
                temperature=CONFIG['temperature']
            )
        except ValueError as e:
            print(f"Skipping window {row['window_id']} due to error in explanation generation: {e}")
            continue

        # Add metadata
        explanation['window_id'] = int(row['window_id'])
        explanation['timestamp'] = datetime.now().isoformat()

        # Save incrementally (in case of interruption)
        f.write(json.dumps(explanation) + '\n')
        f.flush()

        explanations.append(explanation)

print(f"\n✓ Generated {len(explanations)} explanations")
print(f"✓ Saved to: {output_path}")

Generating explanations for 5236 windows...
Output will be saved to: /content/drive/MyDrive/fd-llm/outputs/exp_full_dataset/explanations.jsonl
Progress is saved incrementally (safe to interrupt)



Generating explanations:   0%|          | 0/5236 [00:00<?, ?it/s]


Generating 5 explanations with self-consistency...
  Attempt 1...   ✗ Validation failed: evidence must contain at least 3 numeric claims (found 0)
✗ Invalid (see error above)
  Attempt 2... 

Generating explanations:   0%|          | 0/5236 [00:20<?, ?it/s]


KeyboardInterrupt: 

## Step 7: Summary Statistics


In [ ]:
from collections import Counter

print("="*60)
print("EXPLANATION SUMMARY")
print("="*60)

# Diagnosis distribution
diagnoses = [exp['final_diagnosis'] for exp in explanations]
diagnosis_counts = Counter(diagnoses)

print("\nDiagnosis Distribution:")
for diag, count in diagnosis_counts.most_common():
    pct = count / len(explanations) * 100
    print(f"  {diag}: {count} ({pct:.1f}%)")

# Average metrics
avg_confidence = sum(exp['confidence'] for exp in explanations) / len(explanations)
avg_agreement = sum(exp['meta']['voting_agreement'] for exp in explanations) / len(explanations)

print(f"\nAverage LLM Confidence: {avg_confidence:.3f}")
print(f"Average Voting Agreement: {avg_agreement:.1%}")

# Agreement vs Classifier
classifier_match = sum(
    1 for exp in explanations
    if exp['final_diagnosis'] == exp['meta']['primary_classifier']
)
match_rate = classifier_match / len(explanations)
print(f"LLM-Classifier Agreement: {match_rate:.1%}")


## Step 8: View Example Explanation


In [ ]:
# Show a detailed example
example = explanations[0]

print("="*60)
print("EXAMPLE EXPLANATION")
print("="*60)
print(f"\nWindow ID: {example['window_id']}")
print(f"Final Diagnosis: {example['final_diagnosis']}")
print(f"Confidence: {example['confidence']:.3f}")

print("\nEvidence:")
for i, ev in enumerate(example['evidence'], 1):
    print(f"  {i}. {ev}")

print("\nCross-checks:")
for i, check in enumerate(example['cross_checks'], 1):
    print(f"  {i}. {check}")

print("\nRecommended Actions:")
for i, action in enumerate(example['recommended_actions'], 1):
    print(f"  {i}. {action}")

print(f"\nVoting Agreement: {example['meta']['voting_agreement']:.1%}")
print(f"Classifier Prediction: {example['meta']['primary_classifier']}")


## Step 9: Cleanup (Optional)

Free GPU memory when done.


In [ ]:
# Free GPU memory
llm_explainer.free_memory()

import torch
torch.cuda.empty_cache()

print("✓ GPU memory freed")
print("\nYour explanations are saved to Google Drive:")
print(f"  {CONFIG['output_file']}")


## Download Results (Optional)

If you want to download the explanations directly from Colab instead of accessing via Drive.


In [ ]:
# Download explanations.jsonl to your local machine
from google.colab import files
files.download(CONFIG['output_file'])


---

## Notes & Tips

### Performance
- **T4 (Free tier)**: ~5 sec per window → 5236 windows = ~7 hours
- **V100 (Colab Pro)**: ~3 sec per window → 5236 windows = ~4.5 hours  
- **A100 (Colab Pro+)**: ~2 sec per window → 5236 windows = ~3 hours

### If Session Disconnects
The progress is saved incrementally to Drive. You can:
1. Check the output file in Drive to see how many explanations were completed
2. Reload the predictions file
3. Skip already processed windows:
```python
# Load existing explanations
existing_ids = set()
if output_path.exists():
    with open(output_path, 'r') as f:
        for line in f:
            exp = json.loads(line)
            existing_ids.add(exp['window_id'])

# Filter predictions
pred_df = pred_df[~pred_df['window_id'].isin(existing_ids)]
```

### Testing First
Before running on all 5236 windows, test with a small sample:
```python
CONFIG['max_samples'] = 10  # Process just 10 windows
```

### Next Steps
1. Download `explanations.jsonl` from Google Drive
2. Analyze results locally using pandas
3. Create visualizations and reports
4. Compare LLM explanations with domain expert assessments
